<div style="background-color:	#41e5d3; text-align:center; vertical-align: middle; padding:40px 0; margin-top:30px">
<h1 style="color:white">ML - Verhuuraantallen voorspellen - Heethoofd</h1>
<b style="color:white">Jasper Duncker, Julia Boschman, Nima Agha Ghafar Hamedani</b>
    </div>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

# metrics etc
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
pd.options.display.max_columns = 500

### Bestanden importeren

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
Xtest = test.drop(['date_hour'],axis=1)
train["date_hour"]

0        2011-01-01 00:00:00
1        2011-01-01 01:00:00
2        2011-01-01 02:00:00
3        2011-01-01 03:00:00
4        2011-01-01 04:00:00
                ...         
16632    2012-11-30 19:00:00
16633    2012-11-30 20:00:00
16634    2012-11-30 21:00:00
16635    2012-11-30 22:00:00
16636    2012-11-30 23:00:00
Name: date_hour, Length: 16637, dtype: object

## Opdracht 1: EDA

##### Toon datatypes en basisstatistieken

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16637 entries, 0 to 16636
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date_hour   16637 non-null  object 
 1   holiday     16637 non-null  int64  
 2   weathersit  16637 non-null  int64  
 3   temp        16637 non-null  float64
 4   atemp       16637 non-null  float64
 5   hum         16637 non-null  float64
 6   windspeed   16637 non-null  float64
 7   cnt         16637 non-null  int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 1.0+ MB


In [4]:
train.describe()

,holiday,weathersit,temp,atemp,hum,windspeed,cnt
count,16637.000000,16637.000000,16637.000000,16637.000000,16637.000000,16637.000000,16637.000000
mean,0.028671,1.415580,0.504745,0.482608,0.624756,0.190310,190.477009
std,0.166885,0.637298,0.192369,0.171557,0.193227,0.121915,182.026755
min,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,1.000000,0.340000,0.333300,0.470000,0.104500,41.000000
50%,0.000000,1.000000,0.520000,0.500000,0.620000,0.194000,143.000000
75%,0.000000,2.000000,0.660000,0.621200,0.780000,0.253700,282.000000
max,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,977.000000


In [ ]:
train.isna().sum()

Voor het bestand train hebben we gekeken of het voldoet aan alle voorwaardes om een machine learning model te maken:

1. Het is een DataFrame
2. Het DataFrame heeft geen missende waardes
3. Het DataFrame had één kolom die geen numerieke waardes had en die is wordt omgezet naar een datetime kolom. Deze wordt ook gebruikt om nieuwe (numerieke) kolommen aan te maken.

Dus het bestand voldoet aan de voorwaardes.

In [ ]:
test

In [ ]:
test.info()

In [ ]:
test.describe(method='percentiles')

In [ ]:
test.isna().sum()

Net als bij het bestand train hebben wij gekeken naar de inhoud van het test bestand. Omdat wij deze gebruiken om uiteindelijk ons model mee te testen willen wij dit in de zelfde vorm hebben staan als het train bestand. Zo gaan wij ook in dit bestand de date_hour kolom omzetten naar een datetime en de kolom wordt gebruikt om verschillende kolommen met numerieke waardes aan te maken. Verder voldoet het ook aan alle voorwaardes:

1. Het is een DataFrame.
2. Het DataFrame heeft geen missende waardes.
3. Het DataFrame heeft alleen numerieke waardes.

##### Voeg tijdserie elementen toe en gebruik passende visualisaties om patronen zichtbaar te maken.

In [ ]:
import pandas as pd
train['date_hour'] = pd.to_datetime(train['date_hour'])

# Voeg dag, week, maand, jaar
train['hour_of_day'] = train['date_hour'].dt.hour #Het uur van de dag (0-23).
train['day_of_week'] = train['date_hour'].dt.dayofweek #De dag van de week (maandag is 0, zondag is 6).
train['week'] = train['date_hour'].dt.isocalendar().week #De weken
train['month'] = train['date_hour'].dt.month
train['year'] = train['date_hour'].dt.year
train["day"] = train["date_hour"].dt.day

#Time series / date functionality — pandas 2.1.1 documentation. (n.d.). https://pandas.pydata.org/docs/user_guide/timeseries.html

def get_season(month):
    if month in [3, 4, 5]:  
        return 1
    elif month in [6, 7, 8]:  
        return 2
    elif month in [9, 10, 11]:
        return 3
    else: 
        return 4

train['season'] = train['month'].apply(get_season)

In [ ]:
import pandas as pd
test['date_hour'] = pd.to_datetime(test['date_hour'])

# Voeg dag, week, maand, jaar
test['hour_of_day'] = test['date_hour'].dt.hour #Het uur van de dag (0-23).
test['day_of_week'] = test['date_hour'].dt.dayofweek #De dag van de week (maandag is 0, zondag is 6).
test['week'] = test['date_hour'].dt.isocalendar().week #De weken
test['month'] = test['date_hour'].dt.month
test['year'] = test['date_hour'].dt.year

#Time series / date functionality — pandas 2.1.1 documentation. (n.d.). https://pandas.pydata.org/docs/user_guide/timeseries.html

def get_season(month):
    if month in [3, 4, 5]:  
        return 1
    elif month in [6, 7, 8]:  
        return 2
    elif month in [9, 10, 11]:
        return 3
    else: 
        return 4

test['season'] = test['month'].apply(get_season)

##### Visualisatie 1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


#Invloed van vakantie op het gemiddelde aantal
holiday_data = train[train['holiday'] == 1]
non_holiday_data = train[train['holiday'] == 0]
holiday_avg = holiday_data['cnt'].mean()
non_holiday_avg = non_holiday_data['cnt'].mean()

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.bar(['Feestdag', 'Geen Feestdag'], [holiday_avg, non_holiday_avg])
plt.xlabel('Periode')
plt.ylabel('Gemiddeld aantal')
plt.title('Invloed van vakantie op het gemiddelde aantal')


plt.tight_layout()
plt.show()

### Bevindingen feestdagen

We zien hier dat er gemiddeld minder van het product gebruikt wordt bij een feestdagen.

##### Visualisatie 2

In [ ]:
# Tweede grafiek: Invloed van het uur van de dag op verkoopcijfers
hourly_avg = train.groupby('hour_of_day')['cnt'].mean()
plt.figure(figsize=(16, 5))
plt.subplot(122)
plt.plot(hourly_avg.index, hourly_avg.values, marker='o')
plt.xlabel('Uur van de dag')
plt.ylabel('Gemiddeld aantal verkochte producten')
plt.title('Invloed van het uur van de dag op verkoopcijfers')
plt.xticks(range(24))
plt.grid(True)

# Invloed van de dag van de week op de verkoop

day_of_week_avg = train.groupby('day_of_week')['cnt'].mean()
plt.figure(figsize=(12, 4))
plt.subplot(122)
plt.bar(day_of_week_avg.index, day_of_week_avg.values)
plt.xlabel('Dag van de week')
plt.ylabel('Gemiddeld aantal verkochte producten')
plt.title('Invloed van de dag van de week')

# Invloed van de week op de verkoop
plt.figure(figsize=(16, 6))

weekday_avg = train.groupby('week')['cnt'].mean()
plt.subplot(122)
plt.bar(weekday_avg.index, weekday_avg.values)
plt.xlabel('Week')
plt.ylabel('Gemiddeld aantal verkochte producten')
plt.title('Invloed van de weekdag')

# Invloed van de maand op de verkoop
plt.figure(figsize=(16, 6))

month_avg = train.groupby('month')['cnt'].mean()
plt.subplot(122)
plt.bar(month_avg.index, month_avg.values)
plt.xlabel('Maand')
plt.ylabel('Gemiddeld aantal verkochte producten')
plt.title('Invloed van de maand')

plt.tight_layout()
plt.show()

### Conclusie uur, dag, week, maand

#### Conclusie uur

Wij zien in deze plot dat er 2 pieken vallen. Dat is om 8 uur en om 17 uur. Deze uren vallen ook in de spitsuren. Door deze bevinding kan het product een relatie hebben met vervoer. (DPG Media Privacy Gate, n.d.)

DPG Media Privacy Gate. (n.d.). https://myprivacy.dpgmedia.nl/consent?siteKey=ujm6mv0jrqiz5syr&callbackUrl=https%3A%2F%2Fwww.autoweek.nl%2Fprivacygate-confirm%3FredirectUri%3D%252Fverkeer%252Fzo-vermijd-je-files-en-druk-verkeer-tijdens-de-spits%252F%253Freferrer%253Dhttps%25253A%25252F%25252Fwww.google.com%25252F

#### Conclusie dag

Wij zien dat er op zaterdag (5) en zondag (6) een kleine daling is tussen het gemiddeld gebruik van het product. Hiebij kan je concluderen dat het product te maken heeft met de werkdagen/schooldagen.

#### Conclusie week

We zien dat het bij het einde van de officiële zomervakantie er veel gebruikt van het product gebruikt wordt

#### Conclusie maand
We zien hierbij in de 'warmere maanden'Dat het product vaker gebruikt wordt. Hierdoor kunnen we zeggen dat het een product is dat vaak buiten gebruikt wordt.

##### Onderzoek de relatie tussen de onafhankelijke en de afhankelijke variabelen

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(121)  
heatmap1 = sns.heatmap(train.corr(method='pearson')[['cnt']].sort_values(by='cnt', ascending=False),
                       vmin=-1, vmax=1, annot=True, cmap='RdYlGn')
heatmap1.set_title('Correlatie cnt', fontdict={'fontsize': 12}, pad=12)


plt.subplot(122) 
heatmap2 = sns.heatmap(train.corr(method='pearson'), vmin=-1, vmax=1, annot=True, cmap='RdYlGn')
heatmap2.set_title('Correlatie tussen alle kolommen in train', fontdict={'fontsize': 12}, pad=12)

plt.tight_layout()  
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

temp_avg = train.groupby('temp')['cnt'].mean()
plt.subplot(122)
plt.scatter(temp_avg.index, temp_avg.values)
plt.xlabel('Temperatuur')
plt.ylabel('Gemiddeld aantal verkochte producten')
plt.title('Invloed van de temperatuur')
plt.show()

### Conclusie temperatuur
Uit deze visualisatie zien wij het volgende: Hoe warmer het weer, hoe vaker het product wordt gebruikt. 

In [ ]:
train['datum'] = train['date_hour'].dt.date

train2011 = train[train["year"] == 2011]
train2012 = train[train["year"] == 2012]

plt.hist(train2011["cnt"], alpha = 0.3, color = "purple", edgecolor = 'w', label= '2011')
plt.hist(train2012["cnt"], alpha = 0.4, edgecolor = 'w', label = '2012')
plt.xlabel("Count of product")
plt.ylabel("Amount of times")
plt.title("Count of product in 2011 and 2012")
plt.legend()
plt.show()

In [ ]:
train["year"].value_counts()

In [ ]:
print(train2011["datum"].min(), ",", train2011["datum"].max())

In [ ]:
print(train2012["datum"].min(), ",", train2012["datum"].max())

In [ ]:
print(train2011["cnt"].max())

In [ ]:
print(train2012["cnt"].max())

Het valt ons op dat de data niet helemaal volledig is. Zo zien wij dat er meer datapunten zijn voor het jaar 2011 dan het jaar 2012. Wanneer wij dit verder onderzoeken zien wij dat de maand december ontbreekt in het jaar 2012. Wel zien wij aan de visualisatie dat de waardes van 2012 wel een stuk hoger liggen. Zo zie je in de visualisatie dat in het jaar 2012 waardes liggen tot 977. In 2011 zijn er meerdere waardes die lager liggen. Zo gaat het maximale punt tot 651. Dit is een verschil van eer dan 300.

In [ ]:
plt.figure(figsize=(10, 8))

sns.barplot('year', "cnt",hue='season', data=train, ci=None)
plt.legend(["Spring", "Summer", "Autumn", "Winter"])
plt.xlabel('Year')
plt.ylabel('Total number of count')
plt.title('Number of count per season')
plt.show()



### Conclusie per seizoen
Wij zien dat er in de lente, zomer en herfst meer mensen hebben besloten op het product te gebruiken dan in de winter. Hieruit kan je de hypothese stellen dat het een product is die je buiten gebruikt

In [ ]:
plt.figure(figsize=(10, 8))

sns.barplot('year', "cnt",hue='weathersit', data=train, ci=None)
plt.legend(['Clear', 'Foggy', 'Light snow', 'Heavy rain'])
plt.xlabel('Year')
plt.ylabel('Total number of count')
plt.title('Number of count per weather situation')
plt.show()

### Conclusie per weer situatie
Wij zien dat hoe aantrekkelijker het weer is om naar buiten te gaan, hoe meer er van het product gebruikt wordt. Hieruit kan je de hypothese stellen dat het een product is die je buiten gebruikt

In [ ]:
train

In [ ]:
test

### Metric Functie

In [ ]:
def train_test_model(model):
    """
    Traint het opgegeven model met behulp van de meegeleverde trainingsgegevens en evalueert het met de testgegevens.

    Parameters:
    ----------
    model : model vergelijkbaar met sklearn
        Het model dat getraind en geëvalueerd moet worden.
    
    Returns:
    -------
    RMSE :
        Deze functie drukt de RMSE score van het model op de testgegevens af.
    
        
    """

    model.fit(X_train,y_train)
    y_pred= model.predict(X_test)
    MSE = mean_squared_error(y_test, y_pred)
    RMSE = MSE**0.5
    R2 = r2_score(y_test, y_pred)
    MAE = mean_absolute_error(y_test, y_pred)
    return print('R2:',R2,
                 '\nMAE:', MAE,
                 '\nMSE:',MSE,
                 '\nRMSE:',RMSE)

### TimeSeries

In [ ]:
roll_train = train[["date_hour", "cnt"]].set_index("date_hour")

moving_average = roll_train.rolling(
    window=52,       # 365-day window
    center=True,      # puts the average at the center of the window
    min_periods=26,  # choose about half the window size
).mean()              # compute the mean (could also do median, std, min, max, ...)

ax = roll_train.plot(style=".", color="0.5")
moving_average.plot(
    ax=ax, linewidth=3, title="Renting product - 365-Day Moving Average", legend=False,
);

In [ ]:
moving_average.plot()

In [ ]:
train['datum'] = train['date_hour'].dt.date

nieuw_dataframe = train[['datum', 'cnt']]
nieuw_dataframe.set_index('datum', inplace=True)
nieuw_dataframe.index = pd.to_datetime(nieuw_dataframe.index)

dagelijks_resampled = nieuw_dataframe.resample('D').mean()
weekly_resampled = nieuw_dataframe.resample('W').mean()

#pandas.DataFrame.resample — pandas 2.1.2 documentation. (n.d.). #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html

def calculate_trend(df, window_size):
    """
    Calculates the trend component using a rolling window and moving averages

    Parameters:
        df (pandas.DataFrame): DataFrame with a DateTime index and a single column.
        window_size: The length of the rolling window in units of time
    """
    trend = df.rolling(window_size, center=True).mean()
    return trend

plt.figure(figsize=(10,6))

trend_calculated = calculate_trend(dagelijks_resampled, window_size=12)

plt.plot(trend_calculated, label="Trend calculated", color="red")

plt.title("Trendlijn")
plt.legend()


In [ ]:
seasonal_pattern = weekly_resampled.groupby(weekly_resampled.index.dayofweek)['cnt'].mean()

weekly_resampled['seasonal_adjusted'] = weekly_resampled['cnt'] - weekly_resampled.index.dayofweek.map(seasonal_pattern)

plt.figure(figsize=(10, 6))


plt.plot(weekly_resampled.index, weekly_resampled['seasonal_adjusted'], label='Seizoensgecorrigeerd')
plt.xlabel('Datum')
plt.ylabel('Waarden')
plt.title('Seizoenscomponent visualisatie')
plt.legend()
plt.show()

In [ ]:
train2 = train.copy()
train2 = pd.DataFrame(train2.groupby("datum")["cnt"].sum())
train2

In [ ]:
import statsmodels.api as sm

def decompose_ts(series, period=None):
    decompose = sm.tsa.seasonal_decompose(series, period=period)
    trend = decompose.trend
    seasonal = decompose.seasonal
    resid = decompose.resid
    return trend, seasonal, resid

trend, seasonal, resid = decompose_ts(train2, period= 24)
print(trend, seasonal, resid)

In [ ]:
import matplotlib.pyplot as plt

def plot_ts(df, trend, seasonal, residuals):
    """
    Plots the original time series, trend, seasonal, and residual components.

    Parameters:
        df (pandas.DataFrame): DataFrame with a DateTime index and a single column.
        trend (pandas.Series): Time series representing the trend component.
        seasonal (pandas.Series): Time series representing the seasonal component.
        residuals (pandas.Series): Time series representing the residual component.
    """

    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 8))

    # Plot the original time series and trend
    axes[0].plot(df.index, df.iloc[:, 0], color='blue', label='Original Time Series')
    axes[0].plot(df.index, trend, color='black', linestyle='dashed', label='Trend')
    axes[0].set_title('Original Time Series and Trend')
    axes[0].legend()

    # Plot the seasonal component
    axes[1].plot(df.index, seasonal, color='green', label='Seasonal Component')
    axes[1].set_title('Seasonal Component')
    axes[1].legend()

    # Plot the residual component
    axes[2].plot(df.index, residuals, color='red', label='Residual Component')
    axes[2].set_title('Residual Component')
    axes[2].legend()

    plt.tight_layout()
    plt.show()

plot_ts(train2, trend, seasonal, resid)

In [ ]:
N = len(roll_train)

# Compute the Fourier transform of the timeseries
train_ft = np.fft.fft(roll_train['cnt'])

# Compute the magnitude of the frequencies
magnitude = 2.0/N * np.abs(train_ft[:N//2])

# Plot the constituent frequencies
plt.plot(magnitude)
plt.xlabel('Frequency (1/N)')
plt.ylabel('Amplitude')
plt.title("Frequency domain of rentals timeseries")

plt.tight_layout()
plt.show()

In [ ]:
from scipy.signal import find_peaks
peaks, _ = find_peaks(magnitude, height=45) # Je kunt de "height" parameter bewerken om meer of minder pieken te vinden.

print(f"Peaks found at {peaks}")

plt.plot(magnitude)
plt.xlabel('Frequency (1/N)')
plt.ylabel('Amplitude')
plt.title("Frequency domain of mixture")

plt.plot(peaks, magnitude[peaks], ".", markersize=10)

plt.tight_layout()
plt.show()

De dominante frequenties zijn 2, 692, 694 en 1387. Wij slaan 2 over, omdat deze heel dicht bij de 0 ligt.

In [ ]:
for peak in peaks:
    print(N/peak)

Het valt ons op dat er een aantal dominante frequenties zijn. Wij hebben deze waardes omgerekend naar uren. Wij zien een duidelijk verband op het niveau van een jaar, een dag en een halve dag. Wij gaan een halve dag niet gebruiken, omdat deze waarde geen toevoeging heeft aan ons specifieke onderzoek.

In [ ]:
test = test.set_index('date_hour')
test

In [ ]:
train = train.set_index('date_hour')
train

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier1 = CalendarFourier(freq="A", order=1)  # 1 sin/cos pairs for "A"nnual seasonality
fourier2 = CalendarFourier(freq="D", order=2)  # 1 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index=train.index,
    constant=False,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    seasonal=False,               # weekly seasonality (indicators)
    additional_terms=[fourier1,fourier2],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp.in_sample()  # create features for dates in tunnel.index
X2 = dp.out_of_sample(steps=len(test.index),forecast_index= test.index)
X2

In [ ]:
X['sin(1,freq=A-DEC)'].plot()
X['cos(1,freq=A-DEC)'].plot()

plt.show()

In [ ]:
X['sin(1,freq=D)'].plot()
X['cos(1,freq=D)'].plot()
plt.xlim('2011-01-01','2011-01-05')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(train["temp"], lags = 40, method = "ywm")
plt.title("PACF")
plt.ylim([-0.25, 0.25])

Wij zien in de visualisatie hierboven dat het 6e punt in het blauwe gebied valt. Dit wilt zeggen dat wij voor ons onderzoek gebruik gaan maken van 5 lags. Wij gebruiken deze om een goede voorspelling te maken. Zo staan alle punten buiten het blauwe gebied voor de lags die een goede voorspelling kunnen maken. Wij kiezen voor temperatuur, omdat dit een kolom is die volgens ons veel invloed heeft op het verhuren van het product.

In [ ]:
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(0, lags + lead_time)
        },
        axis=1)

df_lags = make_lags(train['temp'], lags=5)
df_lags = df_lags.fillna(0.0)
df_lags

In [ ]:
cols_to_use = X.columns.difference(train.columns)
cols_to_use

In [ ]:
dfNew = train.merge(X[cols_to_use], left_index=True, right_index=True, how='outer')
dfNew

In [ ]:
dfNew = pd.get_dummies(data = dfNew, columns=['day_of_week','holiday','hour_of_day'])
dfNew

In [ ]:
cols_to_use = df_lags.columns.difference(dfNew.columns)
cols_to_use

In [ ]:
dfNew = dfNew.merge(df_lags[cols_to_use], left_index=True, right_index=True, how='outer')
dfNew

### Maken van testset

In [ ]:
X2

In [ ]:
X2['sin(1,freq=D)'].plot()
X2['cos(1,freq=D)'].plot()
plt.show()

In [ ]:
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(0, lags + lead_time)
        },
        axis=1)

df_lags_test = make_lags(test['temp'], lags=5)
df_lags_test = df_lags_test.fillna(0.0)
df_lags_test

In [ ]:
cols_to_use_1 = X2.columns.difference(test.columns)
df_test = test.merge(X2[cols_to_use_1], left_index=True, right_index=True, how='outer')
df_test = pd.get_dummies(data = df_test, columns=['day_of_week','holiday','hour_of_day'])
cols_to_use_2 = df_lags_test.columns.difference(df_test.columns)
df_test = df_test.merge(df_lags_test[cols_to_use_2], left_index=True, right_index=True, how='outer')
df_test.info()

In [ ]:
dfNew.info()

In [ ]:
cols_to_use_1 = dfNew.columns.difference(df_test.columns)
cols_to_use_1

### Train, test, split

Voor de time series moet er een andere split worden gebruikt dan de train_test_split. We moeten een split doen door cross validation toe te passen of bijvoorbeeld door X[:200] & X[200:900].

In [ ]:
from sklearn.model_selection import train_test_split
X = dfNew.drop(['cnt','datum', 'day', 'holiday_1'],axis=1)
y = dfNew['cnt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=3*7*24, random_state=27, shuffle=False)

### LinearReg

Lineaire regressie is een statistische methode die wordt gebruikt om de relatie tussen twee variabelen te modelleren, waarbij we proberen te voorspellen hoe de ene variabele verandert als de andere verandert. Het doel is om een rechte lijn te vinden die het beste past bij de gegeven datapunten.

Stel je voor dat we de relatie tussen de uren die je besteedt aan studeren (onafhankelijke variabele) en je behaalde cijfer (afhankelijke variabele) willen begrijpen. Lineaire regressie zou proberen een rechte lijn te vinden die het beste past bij de gegeven datapunten, zodat we kunnen voorspellen welk cijfer je zou halen op basis van het aantal uren studie.

De wiskundige vergelijking van een lineaire regressie is meestal iets als:

\[ y = mx + b \]

Waarbij:
- \( y \) is de afhankelijke variabele (bijvoorbeeld je cijfer)
- \( x \) is de onafhankelijke variabele (bijvoorbeeld het aantal uren studie)
- \( m \) is de helling van de lijn
- \( b \) is het snijpunt met de y-as
(What Is Linear Regression?- Spiceworks - Spiceworks, 2023)

Om de beste lijn te vinden, gebruiken we een "loss functie" die de afstand tussen de voorspelde waarden en de werkelijke waarden meet. Het doel is om deze afstand te minimaliseren.
Loss functies worden gebruikt om de prestaties van machine learning-modellen te beoordelen door de voorspelde waarden te vergelijken met de werkelijke waarden. Hier zijn enkele veelgebruikte loss functies voor regressiemodellen:


- Mean Absolute Error (MAE):
Berekent het gemiddelde absolute verschil tussen voorspelde en werkelijke waarden.
 - Voordeel: Eenvoudig te berekenen.
 - Nadeel: Niet differentieerbaar bij nul.

- Mean Squared Error (MSE):
Berekent het gemiddelde gekwadrateerde verschil tussen voorspelde en werkelijke waarden.
 - Voordeel: Efficiënt voor optimalisatiealgoritmen, geeft meer gewicht aan grote fouten.
 - Nadeel: Gevoelig voor outliers.


- Root Mean Squared Error (RMSE):
Wortel van MSE, meet de gemiddelde magnitude van de fouten.
 - Voordeel: Makkelijk te begrijpen.
 - Nadeel: Gevoelig voor outliers, afhankelijk van schaal van de data. (Muniraj, 2022)

Soms voegen we ook "regularisatie" toe aan het model, wat helpt om overmatige complexiteit te voorkomen en overfitting te verminderen. Overfitting treedt op wanneer het model te sterk is aangepast aan de trainingsgegevens en slecht presteert op nieuwe, ongeziene gegevens.

- Wat is regulering in lineaire regressie?
Regulering voegt een straftoekenning toe aan de lossfunctie van een lineair regressiemodel om de
grootte van coëfficiënten of gewichten te verminderen. Er zijn twee soorten regulering: L1 (Lasso) en L2 (Ridge), die coëfficiënten op verschillende manieren beïnvloeden.

- Implementatie van Regulering in Lineaire Regressie:
Gebruik Lasso (L1) en Ridge (L2) klassen in tools zoals scikit-learn in Python. L1 maakt bepaalde coëfficiënten nul (sparse model), terwijl L2 alle coëfficiënten naar nul schuift (maar niet precies naar nul).

- Voordelen en nadelen van regulering in lineaire regressie:
 - Voordelen: Vermindert overaanpassing, verbetert generalisatie, behandelt multicollineariteit.
 - Nadelen: Kan onderaanpassing introduceren, nauwkeurigheid verminderen, en vereist zorgvuldige afstemming en validatie. (How Do You Explain the Impact of Regularization on the Bias-variance Trade-off in Linear Regression?, n.d.)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# Linear Regression
print("Linear:")

lin_reg = LinearRegression()
train_test_model(lin_reg)
lin_reg_pred = lin_reg.predict(X_test)

In [ ]:
lin_pred = lin_reg.predict(df_test)

### Random Forest (Ensemble)

Een Random Forest model is een supervised learning algoritme. Dat wilt zeggen dat met model leert van gelabelde input. Je hebt twee vormen van een Random Forest model. Zo heb je een RandomForestClassfier en een RandomForestRegressor. Het verschil tussen de twee:
 - RandomForestClassfier = voor classificatie data, zoals het voorspellen van diabetes. Hierin is 0 geen diabetes en 1 is wel diabetes.
 - RandomForestRegressor = voor het voorspellen van een regressie. Zo zijn er een hele hoop (continue) waardes die gebruikt worden. Bijvoorbeeld bij het voorspellen van verhuuraantallen. Ze kijken hier naar een aantal en dat is niet zomaar 0 of 1.
    
Een Random Forest is een belangrijk model, omdat het een ensamble is. Dat wilt zeggen dat het meerdere modellen gebruikt om te trainen. Bij een Random Forest is dit een iets ander geval. Hier wordt namelijk één model meerdere keren meegenomen tijdens het trainen. Dit model heet een Decision Tree

De werking van beide modellen zijn ook wat anders. De werking van beide modellen zijn:
 - RandomForestClassfier = Er worden meerdere Decision Trees getraind en daarvan wordt de meest voorkomende waarde voorspeld. Er wordt hier dan ook gebruikt gemaakt van een concept genaamd 'voting'. Als wij dit vertalen krijgen wij het woord stemmen. Makkelijker gezegd kijkt een Random Forest Classifier dus naar de 'meeste stemmen gelden'.
 - RandomForestRegressor = Er worden net als bij de classifier meerdere Decision Trees gebruikt om het model te trainen. Het verschil tussen de twee komt neer op de waardes van de data. Zo kan een regressie model niet werken met stemmen. Omdat alle waardes anders zijn zou hier nooit een waarde uit kunnen komen, omdat alles maar één keer voor komt. Bij een Random Forest Regressor wordt er dus gebruikt gemaakt van het gemiddelde van alle voorspelde punten. 
- De werking van het Decision Tree model wordt hieronder nog uitgebreid uitgelegd.

Ook zijn er een aantal parameters die kunnen worden ingesteld bij het trainen van een Random Forest model:
- n_estimators = de hoeveelheid Decision Trees je wilt gebruiken tijdens het trainen. Bij een classfier is het handig om een oneven aantal te gebruiken. Anders zou het voor kunnen komen dat je een gelijkspel krijgt. Bijvoorbeeld met een n_estimators van 4 kan het zijn dat 2 keer het model 0 aangeeft en 2 keer 1. Om dat te voorkomen kan er bij een classifier beter gebruik worden gemaakt van een oneven aantal. Bij een regressie maakt dit niet uit, omdat er gekeken wordt naar het gemiddelde.
- min_samples_leaf = de diepte van de "boom". Er moet wel goed nagedacht zijn over deze waarde. Zo kan een te kleine waarde leidden tot underfitting en een te hoge waarde juist weer leidden tot overfitting.
    
Er is wel een nadeel van dit model. Omdat het model tijdens het trainen moet kijken naar meerdere Decision Trees duurt het een stuk langer om output te leveren. Dat komt omdat hij alle Decision Tree modellen af moet gaan voor hij een goede voorspelling kan doen.

##### Wiskunde Formule:
RF(x) = 1/N ∑ (N, i=1) hi(x)

Waarbij

- RF = voorspelling van het Random Forest model voor de input van x
- N = het aantal bomen in de Forest (het bos)
- hi = de voorspelling van de i-ste (beginnend bij 1) boom voor de input van x
    
Kortom, het gemiddelde van de som van alle Decision Trees.

(ChatGPT, 2023)

Uit de gridsearch zijn de volgende parameters gekomen:

    param grid:
    {'estimator': squared_error,
     'max_depth': 17,
     'min_samples_split' = 9}

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state = 27, max_depth = 17, min_samples_split = 9)
train_test_model(rf_reg)

In [ ]:
rf_pred = rf_reg.predict(df_test)

### Gradient Boosting

In [ ]:
gb_reg = GradientBoostingRegressor(random_state=27, n_estimators=100, max_depth=5, learning_rate=0.1)
train_test_model(gb_reg)
rf_pred = gb_reg.predict(X_test)

In [ ]:
gb_pred = gb_reg.predict(df_test)

### Decision Tree

Een Decision Tree is een model met een hiërarchie. Zoals de naam al zegt is een Decision Tree net als een boom. Het enige verschil is dat het een omgedraaide boom is in ons model. Zo begin je bij de "Root" (stam) en ga je steeds dieper de "Leafs" (bladeren) in. Er zijn een aantal belangrijke begrippen binnen een Decision Tree:
- Root node = het eerste punt van het model. Hier wordt de eerste splitsing gemaakt. Wordt ook wel gezien als de stam van de boom.
- Branches: de lijnen tussen de nodes (punten) die de nodes opsplitsen.
- Internal nodes = alle punten die nog een laag verder gaan. Dit worden ook wel gezien als nodes met een child node. Denk aan een familie stam boom. Hier gaat het van de ouders (internal nodes) ook door tot de kinderen (child nodes).
- Leaf node = Het diepste punt van de Decision Tree. Opslitsen in kleine subkopjes is niet meer mogelijk. Denk weer aan de boom. Een "Leaf" (blaadje) is het uiterste punt van een boom. Daar groeit niks meer aan.

Een Decision Tree is opgedeeld in een soort vragen systeem. Het geen een waarde aan van een kolom. Heeft het een waarde die lager is dan de gegeven waarde, dan gaat hij naar de ene node. Is de waarde hoger dan de gegeven waarde, dan gaat hij naar de andere node. Dit gebeurt zo lang tot we zijn aangekomen bij de leaf nodes.

Voordelen:
- Je kunt nummeriek en niet nummerieke data in combinatie gebruiken.
- En Decision Tree kan visueel worden weergegeven wat het makkelijker maakt om het te begrijpen en uit te leggen.

Nadelen:
- Het kan instabiel zijn, omdat een kleine verandering kan leiden tot een compleet andere uitslag. zo kunnen er verkeerde voorspellingen worden gedaan die niet realistisch zijn in een samenleving.
- Bij een te diepe "boom" kan er snel sprake zijn van overfitting.
- Bij een te ondiepe "boom" kan er snel sprake zijn van underfitting.

(WallStreetMojo, 2023)

Uit de gridsearch zijn de volgende parameters gekomen:

    param grid:
    {'criterion': friedman_mse,
     'max_depth': 18,
     'min_samples_split' = 8}

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(criterion= 'friedman_mse', min_samples_split = 8, max_depth = 13, random_state = 27)
train_test_model(dt_reg) 

### XGBoost

Support Vector Regression (SVR) is een vorm van Support Vector Machine (SVM) die wordt gebruikt voor het voorspellen van continue uitvoerwaarden, door middel van patronen in data te vinden en voorspellingen te doen op basis van die patronen. Het aspect van 'ondersteunende vectoren' in SVR duidt op de punten die zich het dichtst bij de regressielijn bevinden en een significante invloed uitoefenen op de positie van die lijn. Het hoofddoel is om de modelfouten binnen een vooraf bepaalde tolerantie te handhaven. (ChatGPT, 2023, prompt 1: SVR Model Explained.)

Kernel Types:
SVR kan zowel lineaire als niet-lineaire kernels gebruiken.
 - Lineaire kernel: Eenvoudig inwendig product tussen invoervectoren.
 - Niet-lineaire kernel (bijvoorbeeld 'RBF' - radiale basisfunctie): Kan complexere patronen in de gegevens vastleggen.

In de scikit-learn-bibliotheek voor Python gebruik je de 'SVR'-klasse. 
 - Kernels: SVR gebruikt kernels om gelijkenis tussen invoervectoren te bepalen. Lineaire kernels zijn eenvoudige inwendige producten, terwijl niet-lineaire kernels complexere patronen kunnen vastleggen.
    
 - Hyperparameters: 
    SVR heeft instellingen zoals de 'C'-parameter die het modelgedrag beïnvloeden. Een hogere 'C'-waarde maakt het model strenger, terwijl een lagere waarde toegeeflijker is.
    
(GeeksforGeeks, 2023)

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(colsample_bytree = 0.8, learning_rate = 0.1, max_depth = 7, n_estimators = 200, subsample = 0.5)
train_test_model(xgb_model)

### Hybride model

Zoals de naam al aangeeft is een hybride model een combinatie of kruising tussen twee (of meer) modellen. Het trainen van een hybride model wordt gedaan in een aantal stappen. Zo wordt er eerst een model getraind op de manier dat je gewend bent. In ons voorbeeld trainen wij eerst een Random Forest model. Hiervoor gebruiken wij de zelfde parameters als bij het trainen van ons losse Random Forest model. Hierna trainen wij een Gradient Boosting model. Tijdens het trainen van het Gradient Boosting model wordt er gebruik gemaakt van de uitkomsten van de voorspellingen van het Random Forest model. Ook wordt voor de voorspellingen van het Gradient Boosting model gebruik gemaakt van de voorspellingen met het Random Forest model. 

Dit model zou betere voorspellingen moeten geven, omdat het model uitgaat van de voorspellingen van twee (of meer) modellen. Een punt wat een Hybride model anders maakt dan andere ensemble modellen is het feit dat een Hybride model alleen modellen kunnen gebruiken die hetzelfde probleem willen voorspellen. 

In conclussie gebruik je dus meerdere modellen om tot voorspellingen te komen. Het ene model wordt gebruikt om voor het andere model voorspellingen te doen. Dit werkt anders bij andere modellen waar wij bijvoorbeeld zien dat er meerdere modellen worden gebruikt om tot één antwoord te komen. Dit gebeurt dan om het gemiddelde te pakken tijdens de training. Bij een Hybride model wordt er eerst een model getraind en dat model wordt weer gebruikt om de definitieve voorspellingen op te doen. Dit gaat dus in meerdere stappen. (Wikipedia contributors, 2023)

In [ ]:
# Train een Decision Tree-regressor op de trainingsdata
random_forest = RandomForestRegressor(random_state = 27, max_depth = 17, min_samples_split = 9)
random_forest.fit(X_train, y_train)
random_predictions = random_forest.predict(X_test)
# Gebruik de voorspellingen van de Decision Tree als functies voor een Linear Regression-model
gb_reg = GradientBoostingRegressor(random_state=27, n_estimators=100, max_depth=5, learning_rate=0.1)
gb_reg.fit(random_predictions.reshape(-1, 1), y_test)
# Maak voorspellingen op de testdata met het hybride model
hybrid_predictions = gb_reg.predict(random_predictions.reshape(-1, 1))

# Evalueer de prestaties van het hybride model
rmse = np.sqrt(np.mean((hybrid_predictions - y_test) ** 2))
print(f'Hybride model Root Mean Squared Error: {rmse}')

In [ ]:
random_forest_predictions = random_forest.predict(df_test)
# Use Random Forest predictions as features for Gradient Boosting
hybrid_pred = gb_reg.predict(random_forest_predictions.reshape(-1, 1))

ChatGPT,(2023)

### Pushen Kaggle

In [ ]:
def push_dataframe(p_id_values, predictions_values, push=True, csv_file_path=None):
    """
    Creëert een DataFrame met kolommen 'p_id' en 'predictions' en slaat het op naar een CSV-bestand (optioneel).

    Args:
        p_id_values (list): Een lijst met waarden voor de 'p_id'-kolom.
        predictions_values (list): Een lijst met waarden voor de 'predictions'-kolom.
        push (bool): Een boolean die aangeeft of het DataFrame moet worden opgeslagen naar een CSV-bestand.
        csv_file_path (str): Het pad naar het CSV-bestand waarin het DataFrame moet worden opgeslagen (alleen nodig als push=True).

    Returns:
        pd.DataFrame: Een DataFrame met de opgegeven kolommen.
    """
    
    # Flatten the predictions_values array if it's 2D
    if len(predictions_values.shape) > 1:
        predictions_values = predictions_values.flatten()
        
    data = {'date_hour': p_id_values, 'cnt': predictions_values}
    df = pd.DataFrame(data)
    
    if push and csv_file_path:
        df.to_csv(csv_file_path, index=False)
        print(f"CSV file '{csv_file_path}' has been created.")
    
    return df

In [ ]:
csv_file_path = r'C:\Users\bosch\OneDrive - De Haagse Hogeschool\Bureaublad\ADS&AI\Machine Learning\Modellen Portfolio 2\gb_pred_1.csv'

#parameters
id_values = test.index # niet veranderen
predictions_values = gb_pred #predictions van model

# push functie toepassen
push_dataframe(id_values, predictions_values, push=False, csv_file_path=csv_file_path)

### Literatuurlijst
**Linear Regression:**
- What is Linear Regression?- Spiceworks - Spiceworks. (2023, April 3). Spiceworks. https://www.spiceworks.com/tech/artificial-intelligence/articles/what-is-linear-regression/

- Muniraj, P. (2022, February 25). Loss functions to evaluate Regression Models - Analytics Vidhya - Medium. Medium. https://medium.com/analytics-vidhya/loss-functions-to-evaluate-regression-models-8dac47e327e2

- How do you explain the impact of regularization on the bias-variance trade-off in linear regression? (n.d.). https://www.linkedin.com/advice/0/how-do-you-explain-impact-regularization-bias-variance

**Random Forest:**
- ChatGPT, 2023, Random Forest Regressor Formula. https://chat.openai.com/share/97f92d1a-0bbb-4aef-9269-be6fd4be7234

- AMO. (2022, 4 februari). Het principe van de werking van Random Forest | AMO-PB Icastat. AMO-PB Icastat | advies- en ingenieursbureau modellering, optimalisatie en statistiek. https://www.amo-nl.com/het-principe-van-de-werking-van-random-forest/

**GradientBoosting:**

**Decision Tree:**
- WallStreetMojo, 2023. Decision Tree. https://www.wallstreetmojo.com/decision-tree/

**SVR:**
-  GeeksforGeeks. (2023, January 30). Support Vector Regression  SVR  using Linear and Non Linear Kernels in Scikit Learn. https://www.geeksforgeeks.org/support-vector-regression-svr-using-linear-and-non-linear-kernels-in-scikit-learn/

- ChatGPT, 2023, prompt 1: SVR Model Explained. https://chat.openai.com/share/db240b56-5072-420e-a99c-174cf7d7640a


**Hybride Model:**
- Wikipedia contributors. (2023, 3 februari). Hybrid algorithm. Wikipedia. https://en.wikipedia.org/wiki/Hybrid_algorithm
- ChatGPT, 2023, Hybride Model met TensorFlow. https://chat.openai.com/c/b66a0bdf-da6d-4141-8177-ed8c6ee7ad66